In [ ]:
#| echo: false
#| warning: false
import os

os.environ["KERAS_BACKEND"] = "torch"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch

torch.set_num_threads(1)
torch.set_default_device('cpu')

import matplotlib

import cycler

# colors = ["#91CCCC", "#FF8FA9", "#CC91BC", "#3F9999", "#A5FFB8"]
# colors += ["#4682B4", "#D2691E", "#808080"]
# matplotlib.pyplot.rcParams["axes.prop_cycle"] = cycler.cycler(color=colors)


def set_square_figures():
    matplotlib.pyplot.rcParams["figure.figsize"] = (2.0, 2.0)
    # matplotlib.pyplot.rcParams['figure.figsize'] = (5.0, 3.0)
    # matplotlib.pyplot.rcParams['figure.dpi'] = 350


def set_rectangular_figures():
    matplotlib.pyplot.rcParams["figure.figsize"] = (5.0, 2.0)


set_rectangular_figures()
matplotlib.pyplot.rcParams["figure.dpi"] = 350
matplotlib.pyplot.rcParams["savefig.bbox"] = "tight"
matplotlib.pyplot.rcParams["font.family"] = "serif"

matplotlib.pyplot.rcParams["axes.spines.right"] = False
matplotlib.pyplot.rcParams["axes.spines.top"] = False


def square_fig():
    return matplotlib.pyplot.figure(figsize=(2, 2), dpi=350).gca()


def add_diagonal_line():
    xl = matplotlib.pyplot.xlim()
    yl = matplotlib.pyplot.ylim()

    min_left = min(xl[0], yl[0])
    max_right = max(xl[1], yl[1])
    matplotlib.pyplot.plot(
        [min_left, max_right], [min_left, max_right], color="black", linestyle="--"
    )
    matplotlib.pyplot.xlim([min_left, max_right])
    matplotlib.pyplot.ylim([min_left, max_right])


import pandas

pandas.options.display.max_rows = 6

import numpy

numpy.set_printoptions(precision=2)
numpy.random.seed(123)

# import keras as keras
import keras

keras.utils.set_random_seed(1)

::: {.content-visible unless-format="revealjs"}


In [ ]:
#| code-fold: true
#| code-summary: Show the package imports
import random
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping

:::

# Time Series {visibility="uncounted"}

## Tabular data vs time series data

::: columns
::: column

**Tabular data**

We have a dataset $\{ \boldsymbol{x}_i, y_i \}_{i=1}^n$ which we assume are i.i.d. observations.

| Brand    | Mileage  | # Claims |
|:--------:|:--------:|:--------:|
| BMW      | 101 km   | 1        |
| Audi     | 432 km   | 0        |
| Volvo    | 3 km     | 5        |
| $\vdots$ | $\vdots$ | $\vdots$ |

The goal is to _predict_ the $y$ for some covariates $\boldsymbol{x}$.
:::
::: column
**Time series data**

Have a sequence $\{ \boldsymbol{x}_t, y_t \}_{t=1}^T$ of observations taken at regular time intervals.

| Date  | Humidity | Temp. | 
|:-----:|:--------:|:-----:|
| Jan 1 | 60%      | 20 °C |
| Jan 2 | 65%      | 22 °C | 
| Jan 3 | 70%      | 21 °C |
| $\vdots$ | $\vdots$ | $\vdots$ |


The task is to _forecast_ future values based on the past.

:::
:::

## Attributes of time series data

- **Temporal ordering**: The order of the observations matters.
- **Trend**: The general direction of the data.
- **Noise**: Random fluctuations in the data.
- **Seasonality**: Patterns that repeat at regular intervals.

<!-- - **Autocorrelation**: The correlation of a signal with a delayed copy of itself. -->

::: {.callout-note}
Question: What will be the temperature in Berlin tomorrow?
What information would you use to make a prediction?
:::

## Australian financial stocks {.smaller}


In [ ]:
stocks = pd.read_csv("aus_fin_stocks.csv")
stocks

## Plot


In [ ]:
stocks.plot()

## Data types and NA values

::: columns
::: column


In [ ]:
stocks.info()

:::
::: column


In [ ]:
for col in stocks.columns:
    print(f"{col}: {stocks[col].isna().sum()}")

:::
:::


In [ ]:
asx200 = stocks.pop("ASX200")

## Set the index to the date {.smaller}


In [ ]:
stocks["Date"] = pd.to_datetime(stocks["Date"])
stocks = stocks.set_index("Date") # or `stocks.set_index("Date", inplace=True)`
stocks

## Plot II


In [ ]:
stocks.plot()
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.5), ncol=4);

## Can index using dates I {.smaller}


In [ ]:
stocks.loc["2010-1-4":"2010-01-8"]

::: {.fragment}
Note, these ranges are _inclusive_, not like Python's normal slicing.
:::

## Can index using dates II

So to get 2019's December and all of 2020 for CBA:


In [ ]:
stocks.loc["2019-12":"2020", ["CBA"]]

## Can look at the first differences


In [ ]:
stocks.diff().plot()
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.5), ncol=4);

## Can look at the percentage changes


In [ ]:
stocks.pct_change().plot()
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.5), ncol=4);

## Focus on one stock

::: columns
::: column


In [ ]:
stock = stocks[["CBA"]]
stock

:::
::: column


In [ ]:
stock.plot()

Find first non-missing value


In [ ]:
first_day = stock.dropna().index[0]
first_day

In [ ]:
stock = stock.loc[first_day:]

In [ ]:
stock.isna().sum()

:::
:::


## Fill in the missing values


In [ ]:
missing_day = stock[stock["CBA"].isna()].index[0]
prev_day = missing_day - pd.Timedelta(days=1)
after = missing_day + pd.Timedelta(days=3)

::: columns
::: column


In [ ]:
stock.loc[prev_day:after]

:::
::: column


In [ ]:
stock = stock.ffill()
stock.loc[prev_day:after]

:::
:::


In [ ]:
stock.isna().sum()

# Baseline forecasts

## Persistence forecast

The simplest model is to predict the next value to be the same as the current value.


In [ ]:
stock.loc["2019":, "Persistence"] = stock.loc["2018"].iloc[-1].values[0]
stock.loc["2018-12":"2019"].plot()
plt.axvline("2019", color="black", linestyle="--")

## Trend

We can extrapolate from recent trend:


In [ ]:
past_date = stock.loc["2018"].index[-30]
past = stock.loc[past_date, "CBA"]
latest_date = stock.loc["2018", "CBA"].index[-1]
latest = stock.loc[latest_date, "CBA"]

trend = (latest - past) / (latest_date - past_date).days
print(trend)

tdays_since_cutoff = np.arange(1, len(stock.loc["2019":]) + 1)
stock.loc["2019":, "Trend"] = latest + trend * tdays_since_cutoff

## Trend forecasts


In [ ]:
stock.loc["2018-12":"2019"].plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(ncol=3, loc="upper center", bbox_to_anchor=(0.5, 1.3))

## Which is better?

If we look at the mean squared error (MSE) of the two models:


In [ ]:
persistence_mse = mean_squared_error(stock.loc["2019", "CBA"], stock.loc["2019", "Persistence"])
trend_mse = mean_squared_error(stock.loc["2019", "CBA"], stock.loc["2019", "Trend"])
persistence_mse, trend_mse

## Use the history


In [ ]:
cba_shifted = stock["CBA"].head().shift(1)
both = pd.concat([stock["CBA"].head(), cba_shifted], axis=1, keys=["Today", "Yesterday"])
both

In [ ]:
def lagged_timeseries(df, target, window=30):
    lagged = pd.DataFrame()
    for i in range(window, 0, -1):
        lagged[f"T-{i}"] = df[target].shift(i)
    lagged["T"] = df[target].values
    return lagged

## Lagged time series {.smaller}


In [ ]:
#| echo: false
pandas.options.display.max_rows = 4

In [ ]:
df_lags = lagged_timeseries(stock, "CBA", 40)
df_lags

In [ ]:
#| echo: false
pandas.options.display.max_rows = 6

## Split into training and testing


In [ ]:
# Split the data in time
X_train = df_lags.loc[:"2018"]
X_val = df_lags.loc["2019"]
X_test = df_lags.loc["2020":]

# Remove any with NAs and split into X and y
X_train = X_train.dropna()
X_val = X_val.dropna()
X_test = X_test.dropna()

y_train = X_train.pop("T")
y_val = X_val.pop("T")
y_test = X_test.pop("T")

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

## Inspect the split data {.smaller}


In [ ]:
X_train

## Plot the split


In [ ]:
#| code-fold: true
y_train.plot()
y_val.plot()
y_test.plot()
plt.legend(["Train", "Validation", "Test"]);

## Train on more recent data


In [ ]:
X_train = X_train.loc["2012":]
y_train = y_train.loc["2012":]

In [ ]:
#| code-fold: true
y_train.plot()
y_val.plot()
y_test.plot()
plt.legend(["Train", "Validation", "Test"], loc="center left", bbox_to_anchor=(1, 0.5));

## Rescale by eyeballing it


In [ ]:
X_train = X_train / 100
X_val = X_val / 100
X_test = X_test / 100
y_train = y_train / 100
y_val = y_val / 100
y_test = y_test / 100

In [ ]:
#| code-fold: true
y_train.plot()
y_val.plot()
y_test.plot()
plt.legend(["Train", "Validation", "Test"], loc="center left", bbox_to_anchor=(1, 0.5));

## Fit a linear model


In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train);

Make a forecast for the validation data:


In [ ]:
y_pred = lr.predict(X_val)
stock.loc[X_val.index, "Linear"] = y_pred

In [ ]:
#| code-fold: true
stock.loc["2018-12":"2019"].plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

## Inverse-transform the forecasts


In [ ]:
stock.loc[X_val.index, "Linear"] = 100 * y_pred

In [ ]:
#| code-fold: true
stock.loc["2018-12":"2019"].plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

## Careful with the metrics


In [ ]:
mean_squared_error(y_val, y_pred)

In [ ]:
mean_squared_error(100 * y_val, 100 * y_pred)

In [ ]:
100**2 * mean_squared_error(y_val, y_pred)

In [ ]:
linear_mse = 100**2 * mean_squared_error(y_val, y_pred)
persistence_mse, trend_mse, linear_mse

# Multi-step forecasts

## Comparing apples to apples {.smaller}

The linear model is only producing _one-step-ahead_ forecasts.

The other models are producing _multi-step-ahead_ forecasts.


In [ ]:
stock.loc["2019":, "Shifted"] = stock["CBA"].shift(1).loc["2019":]

In [ ]:
#| code-fold: true
stock.loc["2018-12":"2019"].plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

In [ ]:
shifted_mse = mean_squared_error(stock.loc["2019", "CBA"], stock.loc["2019", "Shifted"])
persistence_mse, trend_mse, linear_mse, shifted_mse

## Autoregressive forecasts

The linear model needs the last 90 days to make a forecast.

**Idea**: Make the first forecast, then use that to make the next forecast, and so on.

$$
\begin{aligned}
    \hat{y}_t &= \beta_0 + \beta_1 y_{t-1} + \beta_2 y_{t-2} + \ldots + \beta_n y_{t-n} \\
    \hat{y}_{t+1} &= \beta_0 + \beta_1 \hat{y}_t + \beta_2 y_{t-1} + \ldots + \beta_n y_{t-n+1} \\
    \hat{y}_{t+2} &= \beta_0 + \beta_1 \hat{y}_{t+1} + \beta_2 \hat{y}_t + \ldots + \beta_n y_{t-n+2}
\end{aligned}
$$
$$\vdots$$
$$
\hat{y}_{t+k} = \beta_0 + \beta_1 \hat{y}_{t+k-1} + \beta_2 \hat{y}_{t+k-2} + \ldots + \beta_n \hat{y}_{t+k-n}
$$

## Autoregressive forecasting function


In [ ]:
def autoregressive_forecast(model, X_val, suppress=False):
    """
    Generate a multi-step forecast using the given model.
    """
    multi_step = pd.Series(index=X_val.index, name="Multi Step")

    # Initialize the input data for forecasting
    input_data = X_val.iloc[0].values.reshape(1, -1)

    for i in range(len(multi_step)):
        # Ensure input_data has the correct feature names
        input_df = pd.DataFrame(input_data, columns=X_val.columns)
        if suppress:
            next_value = model.predict(input_df, verbose=0)
        else:
            next_value = model.predict(input_df) 

        multi_step.iloc[i] = next_value

        # Append that prediction to the input for the next forecast
        if i + 1 < len(multi_step):
            input_data = np.append(input_data[:, 1:], next_value).reshape(1, -1)

    return multi_step

## Look at the autoregressive linear forecasts


In [ ]:
lr_forecast = autoregressive_forecast(lr, X_val)
stock.loc[lr_forecast.index, "MS Linear"] = 100 * lr_forecast

In [ ]:
stock.loc["2018-12":"2019"].drop(["Linear", "Shifted"], axis=1).plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

## Metrics

One-step-ahead forecasts:


In [ ]:
linear_mse, shifted_mse

Multi-step-ahead forecasts:


In [ ]:
multi_step_linear_mse = 100**2 * mean_squared_error(y_val, lr_forecast)
persistence_mse, trend_mse, multi_step_linear_mse

## Prefer only short windows


In [ ]:
stock.loc["2019":"2019-1"].drop(["Linear", "Shifted"], axis=1).plot();
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

> "It's tough to make predictions, especially about the future."

::: footer
Yogi Berra
:::

# Neural network forecasts

## Simple feedforward neural network


In [ ]:
model = Sequential([
        Dense(64, activation="leaky_relu"),
        Dense(1, "softplus")])

model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
if Path("aus_fin_fnn_model.h5").exists():
    model = keras.models.load_model("aus_fin_fnn_model.h5")
else:
    es = EarlyStopping(patience=15, restore_best_weights=True)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500,
        callbacks=[es], verbose=0)
    model.save("aus_fin_fnn_model.h5")

model.summary()

## Forecast and plot


In [ ]:
y_pred = model.predict(X_val, verbose=0)
stock.loc[X_val.index, "FNN"] = 100 * y_pred

In [ ]:
stock.loc["2018-12":"2019"].drop(["Persistence", "Trend", "MS Linear"], axis=1).plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

## Autoregressive forecasts


In [ ]:
nn_forecast = autoregressive_forecast(model, X_val, True)
stock.loc[nn_forecast.index, "MS FNN"] = 100 * nn_forecast

In [ ]:
stock.loc["2018-12":"2019"].drop(["Linear", "Shifted", "FNN"], axis=1).plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

## Metrics

One-step-ahead forecasts:


In [ ]:
nn_mse = 100**2 * mean_squared_error(y_val, y_pred)
linear_mse, shifted_mse, nn_mse

Multi-step-ahead forecasts:


In [ ]:
multi_step_fnn_mse = 100**2 * mean_squared_error(y_val, nn_forecast)
persistence_mse, trend_mse, multi_step_linear_mse, multi_step_fnn_mse

# Recurrent Neural Networks {visibility="uncounted"}

## Basic facts of RNNs

- A recurrent neural network is a type of neural network that is designed to process sequences of data (e.g. time series, sentences).
- A recurrent neural network is any network that contains a recurrent layer.
- A recurrent layer is a layer that processes data in a sequence.
- An RNN can have one or more recurrent layers.
- Weights are shared over time; this allows the model to be used on arbitrary-length sequences.

## Applications

- Forecasting: revenue forecast, weather forecast, predict disease rate from medical history, etc. 
- Classification: given a time series of the activities of a visitor on a website, classify whether the visitor is a bot or a human.
- Event detection: given a continuous data stream, identify the occurrence of a specific event. Example: Detect utterances like "Hey Alexa" from an audio stream.
- Anomaly detection: given a continuous data stream, detect anything unusual happening. Example: Detect unusual activity on the corporate network.

## Origin of the name of RNNs

> A recurrence relation is an equation that expresses each element of a sequence as a function of the preceding ones. More precisely, in the case where only the immediately preceding element is involved, a recurrence relation has the form
> 
> $$ u_n = \psi(n, u_{n-1}) \quad \text{ for } \quad n > 0.$$

__Example__: Factorial $n! = n (n-1)!$ for $n > 0$ given $0! = 1$.

::: footer
Source: Wikipedia, [Recurrence relation](https://en.wikipedia.org/wiki/Recurrence_relation#Definition). 
:::

<!-- 
# Recurrent Cells {visibility="uncounted"}
-->

## Diagram of an RNN cell

The RNN processes each data in the sequence one by one, while keeping memory of what came before.

::: {.content-visible unless-format="revealjs"}
The following figure shows how the recurrent neural network combines an input `X_l` with a preprocessed state of the process `A_l` to produce the output `O_l`. RNNs have a cyclic information processing structure that enables them to pass information sequentially from previous inputs. RNNs can capture dependencies and patterns in sequential data, making them useful for analysing time series data.
:::

![Schematic of a recurrent neural network. E.g. SimpleRNN, LSTM, or GRU.](ISLR2-10_12.svg)

::: footer
Source: James et al (2022), [An Introduction to Statistical Learning](https://www.statlearning.com/), 2nd edition, Figure 10.12.
:::

## A SimpleRNN cell

![Diagram of a SimpleRNN cell.](colah-LSTM3-SimpleRNN.png)

All the outputs before the final one are often discarded.

::: footer
Source: Christopher Olah (2015), [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs), Colah's Blog.
:::

## LSTM internals

::: {.content-visible unless-format="revealjs"}
Simple RNN structures encounter vanishing gradient problems, hence, struggle with learning long term dependencies. LSTM are designed to overcome this problem. LSTMs have a more complex network structure (contains more memory cells and gating mechanisms) and can better regulate the information flow. 
:::

![Diagram of an LSTM cell.](colah-LSTM3-chain.png)
![Notation for the diagram.](colah-LSTM2-notation.png)

::: footer
Source: Christopher Olah (2015), [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs), Colah's Blog.
:::

## GRU internals

::: {.content-visible unless-format="revealjs"}
GRUs are simpler compared to LSTM, hence, computationally more efficient than LSTMs. 
:::

![Diagram of a GRU cell.](colah-LSTM3-var-GRU.png)

::: footer
Source: Christopher Olah (2015), [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs), Colah's Blog.
:::

# Stock prediction with recurrent networks {visibility="uncounted"}

## SimpleRNN


In [ ]:
from keras.layers import SimpleRNN, Reshape
model = Sequential([
        Reshape((-1, 1)),
        SimpleRNN(64, activation="tanh"),
        Dense(1, "softplus")])
model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
#| eval: false
es = EarlyStopping(patience=15, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_val, y_val),
    epochs=500, callbacks=[es], verbose=0)
model.summary()

In [ ]:
#| echo: false
if Path("aus_fin_simplernn_model.h5").exists():
    model = keras.models.load_model("aus_fin_simplernn_model.h5")
else:
    es = EarlyStopping(patience=15, restore_best_weights=True)
    model.fit(X_train, y_train, validation_data=(X_val, y_val),
        epochs=500, callbacks=[es], verbose=0)
    model.save("aus_fin_simplernn_model.h5")
model.summary()

## Forecast and plot


In [ ]:
y_pred = model.predict(X_val.to_numpy(), verbose=0)
stock.loc[X_val.index, "SimpleRNN"] = 100 * y_pred

In [ ]:
#| code-fold: true
stock.loc["2018-12":"2019"].drop(["Persistence", "Trend", "MS Linear", "MS FNN"], axis=1).plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

## Multi-step forecasts


In [ ]:
rnn_forecast = autoregressive_forecast(model, X_val, True)
stock.loc[rnn_forecast.index, "MS RNN"] = 100 * rnn_forecast

In [ ]:
#| code-fold: true
stock.loc["2018-12":"2019"].drop(["Linear", "Shifted", "FNN", "SimpleRNN"], axis=1).plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

## Metrics

One-step-ahead forecasts:


In [ ]:
rnn_mse = 100**2 * mean_squared_error(y_val, y_pred)
linear_mse, shifted_mse, nn_mse, rnn_mse

Multi-step-ahead forecasts:


In [ ]:
multi_step_rnn_mse = 100**2 * mean_squared_error(y_val, rnn_forecast)
persistence_mse, trend_mse, multi_step_linear_mse, multi_step_fnn_mse, multi_step_rnn_mse

## GRU


In [ ]:
from keras.layers import GRU

model = Sequential([Reshape((-1, 1)),
        GRU(16, activation="tanh"),
        Dense(1, "softplus")])
model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
#| eval: false
es = EarlyStopping(patience=15, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_val, y_val),
    epochs=500, callbacks=[es], verbose=0)
model.summary()

In [ ]:
#| echo: false
if Path("aus_fin_gru_model.h5").exists():
    model = keras.models.load_model("aus_fin_gru_model.h5")
else:
    es = EarlyStopping(patience=15, restore_best_weights=True)
    model.fit(X_train, y_train, validation_data=(X_val, y_val),
        epochs=500, callbacks=[es], verbose=0)
    model.save("aus_fin_gru_model.h5")
model.summary()

## Forecast and plot


In [ ]:
y_pred = model.predict(X_val, verbose=0)
stock.loc[X_val.index, "GRU"] = 100 * y_pred

In [ ]:
#| code-fold: true
stock.loc["2018-12":"2019"].drop(["Persistence", "Trend", "MS Linear", "MS FNN", "MS RNN"], axis=1).plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

## Multi-step forecasts


In [ ]:
gru_forecast = autoregressive_forecast(model, X_val, True)
stock.loc[gru_forecast.index, "MS GRU"] = 100 * gru_forecast

In [ ]:
#| code-fold: true
stock.loc["2018-12":"2019"].drop(["Linear", "Shifted", "FNN", "SimpleRNN", "GRU"], axis=1).plot()
plt.axvline("2019", color="black", linestyle="--")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

## Metrics

One-step-ahead forecasts:


In [ ]:
gru_mse = 100**2 * mean_squared_error(y_val, y_pred)
linear_mse, shifted_mse, nn_mse, rnn_mse, gru_mse

Multi-step-ahead forecasts:


In [ ]:
multi_step_gru_mse = 100**2 * mean_squared_error(y_val, gru_forecast)
persistence_mse, trend_mse, multi_step_linear_mse, multi_step_fnn_mse, multi_step_rnn_mse, multi_step_gru_mse

# Internals of the SimpleRNN {visibility="uncounted"}

<!--

## Shapes of data

![Illustration of tensors of different rank.](medium-tensor-rank.png)

::: footer
Source: Paras Patidar (2019), [Tensors — Representation of Data In Neural Networks](https://medium.com/mlait/tensors-representation-of-data-in-neural-networks-bbe8a711b93b), Medium article.
:::


## The `axis` argument in numpy

Starting with a $(3, 4)$-shaped matrix:


In [ ]:
X = np.arange(12).reshape(3, 4)
X

::: {.content-visible unless-format="revealjs"}
The above code creates an array with values from 0 to 11 and converts that array into a matrix with 3 rows and 4 columns.
:::

::: columns
::: column
`axis=0`: $(3, 4) \leadsto (4,)$.


In [ ]:
X.sum(axis=0)

::: {.content-visible unless-format="revealjs"}
The above code returns the column sum. This changes the shape of the matrix from $(3,4)$ to $(4,)$. Similarly, `X.sum(axis=1)` returns row sums and will change the shape of the matrix from $(3,4)$ to $(3,)$.
:::

:::
::: column
`axis=1`: $(3, 4) \leadsto (3,)$.


In [ ]:
X.prod(axis=1)

:::
:::

The return value's rank is one less than the input's rank.

::: {.callout-important}
The `axis` parameter tells us which dimension is removed.
:::

## Using `axis` & `keepdims`

With `keepdims=True`, the rank doesn't change.


In [ ]:
X = np.arange(12).reshape(3, 4)
X

::: columns
::: column
`axis=0`: $(3, 4) \leadsto (1, 4)$.


In [ ]:
X.sum(axis=0, keepdims=True)

:::
::: column
`axis=1`: $(3, 4) \leadsto (3, 1)$.


In [ ]:
X.prod(axis=1, keepdims=True)

:::
:::

::: columns
::: column


In [ ]:
#| error: true
X / X.sum(axis=1)

:::
::: column


In [ ]:
X / X.sum(axis=1, keepdims=True)

:::
:::

-->

## The rank of a time series

Say we had $n$ observations of a time series $x_1, x_2, \dots, x_n$. 

This $\boldsymbol{x} = (x_1, \dots, x_n)$ would have shape $(n,)$ & rank 1.

If instead we had a batch of $b$ time series'

$$
\boldsymbol{X} = \begin{pmatrix}
x_7 & x_8 & \dots & x_{7+n-1} \\
x_2 & x_3 & \dots & x_{2+n-1} \\
\vdots & \vdots & \ddots & \vdots \\
x_3 & x_4 & \dots & x_{3+n-1} \\
\end{pmatrix}  \,,
$$

the batch $\boldsymbol{X}$ would have shape $(b, n)$ & rank 2.

## Multivariate time series

::: {.content-visible unless-format="revealjs"}
Multivariate time series consists of more than 1 variable observation at a given time point. Following example has two variables `x` and `y`. 
:::

::: columns
::: {.column width="35%"}

<center>


In [ ]:
#| echo: false
from IPython.display import Markdown

t = range(4)
x = [f"$x_{i}$" for i in t]
y = [f"$y_{i}$" for i in t]
df = pandas.DataFrame({"$x$": x, "$y$": y})
df.index.name = "$t$"
Markdown(df.to_markdown())

</center>

:::
::: {.column width="65%"}
Say $n$ observations of the $m$ time series, would be a shape $(n, m)$ matrix of rank 2.

In Keras, a batch of $b$ of these time series has shape $(b, n, m)$ and has rank 3.
:::
:::

::: {.callout-note}
Use $\boldsymbol{x}_t \in \mathbb{R}^{1 \times m}$ to denote the vector of all time series at time $t$.
Here, $\boldsymbol{x}_t = (x_t, y_t)$.
:::


## SimpleRNN

Say each prediction is a vector of size $d$, so $\boldsymbol{y}_t \in \mathbb{R}^{1 \times d}$.

Then the main equation of a SimpleRNN, given $\boldsymbol{y}_0 = \boldsymbol{0}$, is

$$ \boldsymbol{y}_t = \psi\bigl( \boldsymbol{x}_t \boldsymbol{W}_x + \boldsymbol{y}_{t-1} \boldsymbol{W}_y + \boldsymbol{b} \bigr) . $$

Here,
$$
\begin{aligned}
&\boldsymbol{x}_t \in \mathbb{R}^{1 \times m}, \boldsymbol{W}_x \in \mathbb{R}^{m \times d}, \\
&\boldsymbol{y}_{t-1} \in \mathbb{R}^{1 \times d}, \boldsymbol{W}_y \in \mathbb{R}^{d \times d}, \text{ and } \boldsymbol{b} \in \mathbb{R}^{d}.
\end{aligned}
$$

::: {.content-visible unless-format="revealjs"}
At each time step, a simple Recurrent Neural Network (RNN) takes an input vector `x_t`, incorporate it with the information from the previous hidden state `{y}_{t-1}` and produces an output vector at each time step `y_t`. The hidden state helps the network remember the context of the previous words, enabling it to make informed predictions about what comes next in the sequence. In a simple RNN, the output at time `(t-1)` is the same as the hidden state at time `t`.
:::

## SimpleRNN (in batches)

::: {.content-visible unless-format="revealjs"}
The difference between RNN and RNNs with batch processing lies in the way how the neural network handles sequences of input data. With batch processing, the model processes multiple ($b$) input sequences simultaneously. The training data is grouped into batches, and the weights are updated based on the average error across the entire batch. Batch processing often results in more stable weight updates, as the model learns from a diverse set of examples in each batch, reducing the impact of noise in individual sequences. 
:::

Say we operate on batches of size $b$, then $\boldsymbol{Y}_t \in \mathbb{R}^{b \times d}$.

The main equation of a SimpleRNN, given $\boldsymbol{Y}_0 = \boldsymbol{0}$, is
$$ \boldsymbol{Y}_t = \psi\bigl( \boldsymbol{X}_t \boldsymbol{W}_x + \boldsymbol{Y}_{t-1} \boldsymbol{W}_y + \boldsymbol{b} \bigr) . $$
Here,
$$
\begin{aligned}
&\boldsymbol{X}_t \in \mathbb{R}^{b \times m}, \boldsymbol{W}_x \in \mathbb{R}^{m \times d}, \\
&\boldsymbol{Y}_{t-1} \in \mathbb{R}^{b \times d}, \boldsymbol{W}_y \in \mathbb{R}^{d \times d}, \text{ and } \boldsymbol{b} \in \mathbb{R}^{d}.
\end{aligned}
$$

::: footer
Remember, $\boldsymbol{X} \in \mathbb{R}^{b \times n \times m}$, $\boldsymbol{Y} \in \mathbb{R}^{b \times d}$, and $\boldsymbol{X}_t$ is equivalent to `X[:, t, :]`.
:::

## Simple Keras demo


In [ ]:
num_obs = 4  # <1>
num_time_steps = 3  # <2>
num_time_series = 2  # <3>

X = (
    np.arange(num_obs * num_time_steps * num_time_series)
    .astype(np.float32)
    .reshape([num_obs, num_time_steps, num_time_series])
)  # <4>

output_size = 1
y = np.array([0, 0, 1, 1])

1. Defines the number of observations
2. Defines the number of time steps
3. Defines the number of time series
4. Reshapes the array to a range 3 tensor (4,3,2)

::: columns
::: column


In [ ]:
X[:2]  # <1>

1. Selects the first two slices along the first dimension. Since the tensor of dimensions (4,3,2), `X[:2]` selects the first two slices (0 and 1) along the first dimension, and returns a sub-tensor of shape (2,3,2). 
:::
::: column


In [ ]:
X[2:]  # <1>

1. Selects the last two slices along the first dimension. The first dimension (axis=0) has size 4. Therefore, `X[2:]`  selects the last two slices (2 and 3) along the first dimension, and returns a sub-tensor of shape (2,3,2). 
:::
:::

## Keras' SimpleRNN

As usual, the `SimpleRNN` is just a layer in Keras. 


In [ ]:
from keras.layers import SimpleRNN  # <1>

random.seed(1234)  # <2>
model = Sequential([SimpleRNN(output_size, activation="sigmoid")])  # <3>
model.compile(loss="binary_crossentropy", metrics=["accuracy"])  # <4>

hist = model.fit(X, y, epochs=500, verbose=False)  # <5>
model.evaluate(X, y, verbose=False)  # <6>

1. Imports the SimpleRNN layer from the Keras library
2. Sets the seed for the random number generator to ensure reproducibility
3. Defines a simple RNN with one output node and sigmoid activation function
4. Specifies binary crossentropy as the loss function (usually used in classification problems), and specifies "accuracy" as the metric to be monitored during training
5. Trains the model for 500 epochs and saves output as `hist`
6. Evaluates the model to obtain a value for the loss and accuracy


The predicted probabilities on the training set are:


In [ ]:
model.predict(X, verbose=0)

## SimpleRNN weights

::: {.content-visible unless-format="revealjs"}
To verify the results of predicted probabilities, we can obtain the weights of the fitted model and calculate the outcome manually as follows.
:::


In [ ]:
model.get_weights()

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


W_x, W_y, b = model.get_weights()

Y = np.zeros((num_obs, output_size), dtype=np.float32)
for t in range(num_time_steps):
    X_t = X[:, t, :]
    z = X_t @ W_x + Y @ W_y + b
    Y = sigmoid(z)

Y

# Other recurrent network variants {visibility="uncounted"}

## Input and output sequences

![Categories of recurrent neural networks: sequence to sequence, sequence to vector, vector to sequence, encoder-decoder network.](Geron-rnnType.png)

::: footer
Source: Aurélien Géron (2019), _Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow_, 2nd Edition, Chapter 15.
:::


## Input and output sequences

- Sequence to sequence: Useful for predicting time series such as using prices over the last $N$ days to output the prices shifted one day into the future (i.e. from $N-1$ days ago to tomorrow.)
- Sequence to vector: ignore all outputs in the previous time steps except for the last one. Example: give a sentiment score to a sequence of words corresponding to a movie review.

## Input and output sequences

- Vector to sequence: feed the network the same input vector over and over at each time step and let it output a sequence. Example: given that the input is an image, find a caption for it. The image is treated as an input vector (pixels in an image do not follow a sequence). The caption is a sequence of textual description of the image. A dataset containing images and their descriptions is the input of the RNN.
- The Encoder-Decoder: The encoder is a sequence-to-vector network. The decoder is a vector-to-sequence network. Example: Feed the network a sequence in one language. Use the encoder to convert the sentence into a single vector representation. The decoder decodes this vector into the translation of the sentence in another language.

## Recurrent layers can be stacked.

![_Deep RNN_ unrolled through time.](Geron-recurrentneurondeep.png)


::: footer
Source: Aurélien Géron (2019), _Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow_, 2nd Edition, Chapter 15.
:::


# CoreLogic Hedonic Home Value Index {visibility="uncounted"}

## Australian House Price Indices


In [ ]:
#|echo: false
house_prices = pd.read_csv("sa3-capitals-house-price-index.csv", index_col=0)
house_prices.index = pd.to_datetime(house_prices.index)
house_prices.index.name = "Date"
house_prices = house_prices[house_prices.index > pd.to_datetime("1990")]
house_prices.columns = [
    "Brisbane",
    "East_Bris",
    "North_Bris",
    "West_Bris",
    "Melbourne",
    "North_Syd",
    "Sydney",
]
house_prices.plot(legend=False);

::: {.callout-note}
I apologise in advance for not being able to share this dataset with anyone (it is not mine to share).
:::

## Percentage changes {.smaller}


In [ ]:
changes = house_prices.pct_change().dropna()
changes.round(2)

In [ ]:
#| echo: false
pandas.options.display.max_rows = 7

## Percentage changes 


In [ ]:
#| eval: false
changes.plot();

In [ ]:
#| echo: false
changes.plot(lw=1)
matplotlib.pyplot.legend(
    bbox_to_anchor=(0.5, 1.0), loc="lower center", frameon=False, ncol=3
);

## The size of the changes

::: columns
::: column

In [ ]:
changes.mean()

In [ ]:
changes *= 100

In [ ]:
changes.mean()

:::
::: column

In [ ]:
#| echo: false
# set_square_figures() 

In [ ]:
changes.plot(legend=False);

:::
:::


## Split _without_ shuffling


In [ ]:
num_train = int(0.6 * len(changes))
num_val = int(0.2 * len(changes))
num_test = len(changes) - num_train - num_val
print(f"# Train: {num_train}, # Val: {num_val}, # Test: {num_test}")

In [ ]:
#| echo: false
changes.iloc[:num_train, 0].plot(c="tab:blue", lw=1, alpha=0.5)
changes.iloc[num_train : (num_train + num_val), 0].plot(
    c="tab:orange", ax=plt.gca(), lw=1, alpha=0.5
)
changes.iloc[(num_train + num_val) :, 0].plot(c="tab:green", ax=plt.gca(), lw=1, alpha=0.5)
changes.iloc[:num_train, 1:].plot(c="tab:blue", ax=plt.gca(), lw=1, alpha=0.5)
changes.iloc[num_train : (num_train + num_val), 1:].plot(
    c="tab:orange", ax=plt.gca(), lw=1, alpha=0.5
)
changes.iloc[(num_train + num_val) :, 1:].plot(c="tab:green", ax=plt.gca(), lw=1, alpha=0.5)
plt.legend(["Train", "Val", "Test"], frameon=False, ncol=3);

In [ ]:
#| echo: false
set_rectangular_figures() 

## Subsequences of a time series

Keras has a built-in method for converting a time series into subsequences/chunks.


In [ ]:
#| warning: false
from keras.utils import timeseries_dataset_from_array

integers = range(10)
dummy_dataset = timeseries_dataset_from_array(
    data=integers[:-3],
    targets=integers[3:],
    sequence_length=3,
    batch_size=2,
)

for inputs, targets in dummy_dataset:
    for i in range(inputs.shape[0]):
        print([int(x) for x in inputs[i]], int(targets[i]))

::: footer
Source: Code snippet in Chapter 10 of Chollet.
:::


<!-- 
## On time series splits

If you have a lot of time series data, then use:

```python
from keras.utils import timeseries_dataset_from_array

data = range(20)
seq = 3
ts = data[:-seq]
target = data[seq:]
num_train = int(0.5 * len(ts))
num_val = int(0.25 * len(ts))
num_test = len(ts) - num_train - num_val
print(f"# Train: {num_train}, # Val: {num_val}, # Test: {num_test}")
```

::: columns
::: {.column width="33%"}

```python
train_ds = timeseries_dataset_from_array(ts, target, seq, end_index=num_train)
```

:::
::: {.column width="33%"}

```python
val_ds = timeseries_dataset_from_array(
    ts, target, seq, start_index=num_train, end_index=num_train + num_val
)
```

:::
::: {.column width="33%"}

```python
test_ds = timeseries_dataset_from_array(ts, target, seq, start_index=num_train + num_val)
```

:::
:::

::: columns
::: {.column width="33%"}

```python
#| echo: false
print("Training dataset")
for inputs, targets in train_ds:
    for i in range(inputs.shape[0]):
        print([int(x) for x in inputs[i]], int(targets[i]))
```

:::
::: {.column width="33%"}

```python
#| echo: false
print("Validation dataset")
for inputs, targets in val_ds:
    for i in range(inputs.shape[0]):
        print([int(x) for x in inputs[i]], int(targets[i]))
```

:::
::: {.column width="33%"}

```python
#| echo: false
print("Test dataset")
for inputs, targets in test_ds:
    for i in range(inputs.shape[0]):
        print([int(x) for x in inputs[i]], int(targets[i]))
```

:::
:::

::: footer
Adapted from: François Chollet (2021), _Deep Learning with Python_, Second Edition, Listing 10.7.
:::

## On time series splits II

If you _don't_ have a lot of time series data, consider:

```python
X = []
y = []
for i in range(len(data) - seq):
    X.append(data[i : i + seq])
    y.append(data[i + seq])
X = np.array(X)
y = np.array(y);
```

::: columns
::: {.column width="33%"}

```python
num_train = int(0.5 * X.shape[0])
X_train = X[:num_train]
y_train = y[:num_train]
```

:::
::: {.column width="33%"}

```python
num_val = int(np.ceil(0.25 * X.shape[0]))
X_val = X[num_train : num_train + num_val]
y_val = y[num_train : num_train + num_val]
```

:::
::: {.column width="33%"}

```python
num_test = X.shape[0] - num_train - num_val
X_test = X[num_train + num_val :]
y_test = y[num_train + num_val :]
```

:::
:::

::: columns
::: {.column width="33%"}

```python
#| echo: false
print("Training dataset")
for i in range(X_train.shape[0]):
    print([int(x) for x in X_train[i]], int(y_train[i]))
```

:::
::: {.column width="33%"}

```python
#| echo: false
print("Validation dataset")
for i in range(X_val.shape[0]):
    print([int(x) for x in X_val[i]], int(y_val[i]))
```

:::
::: {.column width="33%"}

```python
#| echo: false
print("Test dataset")
for i in range(X_test.shape[0]):
    print([int(x) for x in X_test[i]], int(y_test[i]))
```

:::
:::
-->

# Predicting Sydney House Prices {visibility="uncounted"}

## Creating dataset objects

::: columns
::: column

In [ ]:
# Num. of input time series.
num_ts = changes.shape[1]

# How many prev. months to use.
seq_length = 6

# Predict the next month ahead.
ahead = 1

# The index of the first target.
delay = seq_length + ahead - 1

:::
::: column

In [ ]:
# Which suburb to predict.
target_suburb = changes["Sydney"]

train_ds = timeseries_dataset_from_array(
    changes[:-delay],
    targets=target_suburb[delay:],
    sequence_length=seq_length,
    end_index=num_train,
)

:::
:::

::: columns
::: column

In [ ]:
val_ds = timeseries_dataset_from_array(
    changes[:-delay],
    targets=target_suburb[delay:],
    sequence_length=seq_length,
    start_index=num_train,
    end_index=num_train + num_val,
)

:::
::: column

In [ ]:
test_ds = timeseries_dataset_from_array(
    changes[:-delay],
    targets=target_suburb[delay:],
    sequence_length=seq_length,
    start_index=num_train + num_val,
)

:::
:::

## Converting `Dataset` to numpy

The `Dataset` object can be handed to Keras directly, but if we really need a numpy array, we can run:

In [ ]:
X_train = np.concatenate(list(train_ds.map(lambda x, y: x)))
y_train = np.concatenate(list(train_ds.map(lambda x, y: y)))

The shape of our training set is now:


In [ ]:
X_train.shape

In [ ]:
y_train.shape

Converting the rest to numpy arrays:


In [ ]:
X_val = np.concatenate(list(val_ds.map(lambda x, y: x)))
y_val = np.concatenate(list(val_ds.map(lambda x, y: y)))
X_test = np.concatenate(list(test_ds.map(lambda x, y: x)))
y_test = np.concatenate(list(test_ds.map(lambda x, y: y)))

## A dense network


In [ ]:
from keras.layers import Input, Flatten
random.seed(1)
model_dense = Sequential([
    Input((seq_length, num_ts)),
    Flatten(),
    Dense(50, activation="leaky_relu"),
    Dense(20, activation="leaky_relu"),
    Dense(1, activation="linear")
])
model_dense.compile(loss="mse", optimizer="adam")
print(f"This model has {model_dense.count_params()} parameters.")

es = EarlyStopping(patience=50, restore_best_weights=True, verbose=1)
%time hist = model_dense.fit(X_train, y_train, epochs=1_000, \
  validation_data=(X_val, y_val), callbacks=[es], verbose=0);

## Plot the model

In [ ]:
from keras.utils import plot_model

plot_model(model_dense, show_shapes=True)

## Assess the fits


In [ ]:
model_dense.evaluate(X_val, y_val, verbose=0)

In [ ]:
#| code-fold: true
y_pred = model_dense.predict(X_val, verbose=0)
plt.plot(y_val, label="Sydney")
plt.plot(y_pred, label="Dense")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

## A `SimpleRNN` layer


In [ ]:
random.seed(1)

model_simple = Sequential([
    Input((seq_length, num_ts)),
    SimpleRNN(50),
    Dense(1, activation="linear")
])
model_simple.compile(loss="mse", optimizer="adam")
print(f"This model has {model_simple.count_params()} parameters.")

es = EarlyStopping(patience=50, restore_best_weights=True, verbose=1)
%time hist = model_simple.fit(X_train, y_train, epochs=1_000, \
  validation_data=(X_val, y_val), callbacks=[es], verbose=0);

## Plot the model

In [ ]:
plot_model(model_simple, show_shapes=True)

## Assess the fits


In [ ]:
model_simple.evaluate(X_val, y_val, verbose=0)

In [ ]:
#| code-fold: true
y_pred = model_simple.predict(X_val, verbose=0)

plt.plot(y_val, label="Sydney")
plt.plot(y_pred, label="SimpleRNN")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

## A `LSTM` layer


In [ ]:
from keras.layers import LSTM

random.seed(1)

model_lstm = Sequential([
    Input((seq_length, num_ts)),
    LSTM(50),
    Dense(1, activation="linear")
])

model_lstm.compile(loss="mse", optimizer="adam")

es = EarlyStopping(patience=50, restore_best_weights=True, verbose=1)

%time hist = model_lstm.fit(X_train, y_train, epochs=1_000, \
  validation_data=(X_val, y_val), callbacks=[es], verbose=0);

## Assess the fits


In [ ]:
model_lstm.evaluate(X_val, y_val, verbose=0)

In [ ]:
#| code-fold: true
y_pred = model_lstm.predict(X_val, verbose=0)
plt.plot(y_val, label="Sydney")
plt.plot(y_pred, label="LSTM")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

## A `GRU` layer


In [ ]:
from keras.layers import GRU

random.seed(1)

model_gru = Sequential([
    Input((seq_length, num_ts)),
    GRU(50),
    Dense(1, activation="linear")
])

model_gru.compile(loss="mse", optimizer="adam")

es = EarlyStopping(patience=50, restore_best_weights=True, verbose=1)

%time hist = model_gru.fit(X_train, y_train, epochs=1_000, \
  validation_data=(X_val, y_val), callbacks=[es], verbose=0)

## Assess the fits


In [ ]:
model_gru.evaluate(X_val, y_val, verbose=0)

In [ ]:
#| code-fold: true
y_pred = model_gru.predict(X_val, verbose=0)
plt.plot(y_val, label="Sydney")
plt.plot(y_pred, label="GRU")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

## Two `GRU` layers


In [ ]:
random.seed(1)

model_two_grus = Sequential([
    Input((seq_length, num_ts)),
    GRU(50, return_sequences=True),
    GRU(50),
    Dense(1, activation="linear")
])

model_two_grus.compile(loss="mse", optimizer="adam")

es = EarlyStopping(patience=50, restore_best_weights=True, verbose=1)

%time hist = model_two_grus.fit(X_train, y_train, epochs=1_000, \
  validation_data=(X_val, y_val), callbacks=[es], verbose=0)

## Assess the fits


In [ ]:
model_two_grus.evaluate(X_val, y_val, verbose=0)

In [ ]:
#| code-fold: true
y_pred = model_two_grus.predict(X_val, verbose=0)
plt.plot(y_val, label="Sydney")
plt.plot(y_pred, label="2 GRUs")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

## Compare the models


In [ ]:
#| echo: false
models = [model_dense, model_simple, model_lstm, model_gru, model_two_grus]
model_names = ["Dense", "SimpleRNN", "LSTM", "GRU", "2 GRUs"]
mse_val = {
    name: model.evaluate(X_val, y_val, verbose=0)
    for name, model in zip(model_names, models)
}
val_results = pd.DataFrame({"Model": mse_val.keys(), "MSE": mse_val.values()})
val_results.sort_values("MSE", ascending=False)

The network with two GRU layers is the best. 


In [ ]:
model_two_grus.evaluate(test_ds, verbose=0)

## Test set


In [ ]:
#| code-fold: true
y_pred = model_two_grus.predict(test_ds, verbose=0)
plt.plot(y_test, label="Sydney")
plt.plot(y_pred, label="2 GRU")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

# Predicting Multiple Time Series {visibility="uncounted"}

## Creating dataset objects

::: columns
::: column
Change the `targets` argument to include all the suburbs.
:::
::: column

In [ ]:
train_ds = timeseries_dataset_from_array(
    changes[:-delay],
    targets=changes[delay:],
    sequence_length=seq_length,
    end_index=num_train,
)

:::
:::

::: columns
::: column

In [ ]:
val_ds = timeseries_dataset_from_array(
    changes[:-delay],
    targets=changes[delay:],
    sequence_length=seq_length,
    start_index=num_train,
    end_index=num_train + num_val,
)

:::
::: column

In [ ]:
test_ds = timeseries_dataset_from_array(
    changes[:-delay],
    targets=changes[delay:],
    sequence_length=seq_length,
    start_index=num_train + num_val,
)

:::
:::

## Converting `Dataset` to numpy

The shape of our training set is now:


In [ ]:
X_train = np.concatenate(list(train_ds.map(lambda x, y: x)))
X_train.shape

In [ ]:
y_train = np.concatenate(list(train_ds.map(lambda x, y: y)))
y_train.shape

Converting the rest to numpy arrays:


In [ ]:
X_val = np.concatenate(list(val_ds.map(lambda x, y: x)))
y_val = np.concatenate(list(val_ds.map(lambda x, y: y)))
X_test = np.concatenate(list(test_ds.map(lambda x, y: x)))
y_test = np.concatenate(list(test_ds.map(lambda x, y: y)))

## A dense network


In [ ]:
random.seed(1)
model_dense = Sequential([
    Input((seq_length, num_ts)),
    Flatten(),
    Dense(50, activation="leaky_relu"),
    Dense(20, activation="leaky_relu"),
    Dense(num_ts, activation="linear")
])
model_dense.compile(loss="mse", optimizer="adam")
print(f"This model has {model_dense.count_params()} parameters.")

es = EarlyStopping(patience=50, restore_best_weights=True, verbose=1)
%time hist = model_dense.fit(X_train, y_train, epochs=1_000, \
  validation_data=(X_val, y_val), callbacks=[es], verbose=0);

## Plot the model

In [ ]:
plot_model(model_dense, show_shapes=True)

## Assess the fits


In [ ]:
model_dense.evaluate(X_val, y_val, verbose=0)

::: columns
::: column

In [ ]:
#| code-fold: true
Y_pred = model_dense.predict(X_val, verbose=0)
plt.scatter(y_val, Y_pred)
add_diagonal_line()
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

plt.plot(y_val[:, 4], label="Melbourne")
plt.plot(Y_pred[:, 4], label="Dense")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
::: column

In [ ]:
#| code-fold: true
plt.plot(y_val[:, 0], label="Brisbane")
plt.plot(Y_pred[:, 0], label="Dense")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False)
plt.show()

plt.plot(y_val[:, 6], label="Sydney")
plt.plot(Y_pred[:, 6], label="Dense")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
:::

## A `SimpleRNN` layer


In [ ]:
random.seed(1)

model_simple = Sequential([
    Input((seq_length, num_ts)),
    SimpleRNN(50),
    Dense(num_ts, activation="linear")
])
model_simple.compile(loss="mse", optimizer="adam")
print(f"This model has {model_simple.count_params()} parameters.")

es = EarlyStopping(patience=50, restore_best_weights=True, verbose=1)
%time hist = model_simple.fit(X_train, y_train, epochs=1_000, \
  validation_data=(X_val, y_val), callbacks=[es], verbose=0);

## Plot the model

In [ ]:
plot_model(model_simple, show_shapes=True)

## Assess the fits


In [ ]:
model_simple.evaluate(X_val, y_val, verbose=0)

::: columns
::: column

In [ ]:
#| code-fold: true
Y_pred = model_simple.predict(X_val, verbose=0)
plt.scatter(y_val, Y_pred)
add_diagonal_line()
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

plt.plot(y_val[:, 4], label="Melbourne")
plt.plot(Y_pred[:, 4], label="SimpleRNN")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
::: column

In [ ]:
#| code-fold: true
plt.plot(y_val[:, 0], label="Brisbane")
plt.plot(Y_pred[:, 0], label="SimpleRNN")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False)
plt.show()

plt.plot(y_val[:, 6], label="Sydney")
plt.plot(Y_pred[:, 6], label="SimpleRNN")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
:::


## A `LSTM` layer


In [ ]:
random.seed(1)

model_lstm = Sequential([
    Input((seq_length, num_ts)),
    LSTM(50),
    Dense(num_ts, activation="linear")
])

model_lstm.compile(loss="mse", optimizer="adam")

es = EarlyStopping(patience=50, restore_best_weights=True, verbose=1)

%time hist = model_lstm.fit(X_train, y_train, epochs=1_000, \
  validation_data=(X_val, y_val), callbacks=[es], verbose=0);

## Assess the fits


In [ ]:
model_lstm.evaluate(X_val, y_val, verbose=0)

::: columns
::: column

In [ ]:
#| code-fold: true
Y_pred = model_lstm.predict(X_val, verbose=0)
plt.scatter(y_val, Y_pred)
add_diagonal_line()
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

plt.plot(y_val[:, 4], label="Melbourne")
plt.plot(Y_pred[:, 4], label="LSTM")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
::: column

In [ ]:
#| code-fold: true
plt.plot(y_val[:, 0], label="Brisbane")
plt.plot(Y_pred[:, 0], label="LSTM")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False)
plt.show()

plt.plot(y_val[:, 6], label="Sydney")
plt.plot(Y_pred[:, 6], label="LSTM")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
:::

## A `GRU` layer


In [ ]:
random.seed(1)

model_gru = Sequential([
    Input((seq_length, num_ts)),
    GRU(50),
    Dense(num_ts, activation="linear")
])

model_gru.compile(loss="mse", optimizer="adam")

es = EarlyStopping(patience=50, restore_best_weights=True, verbose=1)

%time hist = model_gru.fit(X_train, y_train, epochs=1_000, \
  validation_data=(X_val, y_val), callbacks=[es], verbose=0)

## Assess the fits


In [ ]:
model_gru.evaluate(X_val, y_val, verbose=0)

::: columns
::: column

In [ ]:
#| code-fold: true
Y_pred = model_gru.predict(X_val, verbose=0)
plt.scatter(y_val, Y_pred)
add_diagonal_line()
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

plt.plot(y_val[:, 4], label="Melbourne")
plt.plot(Y_pred[:, 4], label="GRU")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
::: column

In [ ]:
#| code-fold: true
plt.plot(y_val[:, 0], label="Brisbane")
plt.plot(Y_pred[:, 0], label="GRU")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False)
plt.show()

plt.plot(y_val[:, 6], label="Sydney")
plt.plot(Y_pred[:, 6], label="GRU")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
:::

## Two `GRU` layers


In [ ]:
random.seed(1)

model_two_grus = Sequential([
    Input((seq_length, num_ts)),
    GRU(50, return_sequences=True),
    GRU(50),
    Dense(num_ts, activation="linear")
])

model_two_grus.compile(loss="mse", optimizer="adam")

es = EarlyStopping(patience=50, restore_best_weights=True, verbose=1)

%time hist = model_two_grus.fit(X_train, y_train, epochs=1_000, \
  validation_data=(X_val, y_val), callbacks=[es], verbose=0)

## Assess the fits


In [ ]:
model_two_grus.evaluate(X_val, y_val, verbose=0)

::: columns
::: column

In [ ]:
#| code-fold: true
Y_pred = model_two_grus.predict(X_val, verbose=0)
plt.scatter(y_val, Y_pred)
add_diagonal_line()
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

plt.plot(y_val[:, 4], label="Melbourne")
plt.plot(Y_pred[:, 4], label="2 GRUs")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
::: column

In [ ]:
#| code-fold: true
plt.plot(y_val[:, 0], label="Brisbane")
plt.plot(Y_pred[:, 0], label="2 GRUs")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False)
plt.show()

plt.plot(y_val[:, 6], label="Sydney")
plt.plot(Y_pred[:, 6], label="2 GRUs")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
:::

## Compare the models


In [ ]:
#| code-fold: true
models = [model_dense, model_simple, model_lstm, model_gru, model_two_grus]
model_names = ["Dense", "SimpleRNN", "LSTM", "GRU", "2 GRUs"]
mse_val = {
    name: model.evaluate(X_val, y_val, verbose=0)
    for name, model in zip(model_names, models)
}
val_results = pd.DataFrame({"Model": mse_val.keys(), "MSE": mse_val.values()})
val_results.sort_values("MSE", ascending=False)

The network with an LSTM layer is the best. 


In [ ]:
model_lstm.evaluate(test_ds, verbose=0)

## Test set

::: columns
::: column

In [ ]:
#| code-fold: true
Y_pred = model_lstm.predict(test_ds, verbose=0)
plt.scatter(y_test, Y_pred)
add_diagonal_line()
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

plt.plot(y_test[:, 4], label="Melbourne")
plt.plot(Y_pred[:, 4], label="GRU")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
::: column

In [ ]:
#| code-fold: true
plt.plot(y_test[:, 0], label="Brisbane")
plt.plot(Y_pred[:, 0], label="GRU")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False)
plt.show()

plt.plot(y_test[:, 6], label="Sydney")
plt.plot(Y_pred[:, 6], label="GRU")
plt.xlabel("Time")
plt.ylabel("Change in HPI (%)")
plt.legend(frameon=False);

:::
:::



## Package Versions {.appendix data-visibility="uncounted"}


In [ ]:
from watermark import watermark
print(watermark(python=True, packages="keras,matplotlib,numpy,pandas,seaborn,scipy,torch,tensorflow,tf_keras"))

## Glossary {.appendix data-visibility="uncounted"}

- autoregressive forecasting
- forecasting
- GRU
- LSTM
- one-step/multi-step ahead forecasting
- persistence forecast
- recurrent neural networks
- SimpleRNN